# Real Estate Scrape

In [1]:
import re
import time
import requests
import gmaps.datasets
import pandas as pd
import numpy as np
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from config import username, password, api_key

### Login Automation

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

rand_short = np.random.uniform(0.000, 3.000, size = 5) 
rand_long = np.random.uniform(0.000, 9.000, size = 4) 

url = 'https://www.recolorado.com/' # visit real estate website
browser.visit(url)
time.sleep(rand_short[0])

browser.find_by_tag('a')[63].click() # login process, click 
time.sleep(rand_short[1])

browser.fill('loginid', username) # username 
time.sleep(rand_short[2])

browser.fill('password', password) # password 
time.sleep(rand_short[3])

browser.find_by_value('login').click() # click button
time.sleep(rand_long[0])
browser.windows[1].close_others() # close other windows



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/brandon/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


In [3]:
time.sleep(rand_long[1] + rand_long[2])
mls_app = browser.find_by_xpath('//*[@id="appColumn99"]') # table_app.is_empty()
mls_app.click()
time.sleep(rand_short[4])
browser.windows[1].close_others() # close other windows

### Active Listings

In [4]:
# navigate from homepage to active listings
active_listings = browser.find_by_id('m_wm_w4_m_lv_ctrl0_m_lnkWatch')
active_listings.click()

In [5]:
Active_df_table = {'listing_id':[], 'city':[], 'address':[], 'property_type':[], 
                'price':[], 'total_square_ft':[], 'above_square_ft':[], 
                'bed':[], 'bath':[], 'list_date':[], 'hoa':[]}

time.sleep(3)
Active_soup = soup(browser.html, 'html.parser')
pages = Active_soup.find_all('span', class_='pagingLinks')[1].get_text()
pages = re.search('...(.*)N',pages)
pages = int(pages.group(1))

page_count = 10

In [6]:
for page in range(pages):
    Active_soup = soup(browser.html, 'html.parser')
    Active_table = Active_soup.find('table',
                                    class_='displayGrid nonresponsive ajax_display d26m_show clickableDisplay')
    Active_body = Active_table.tbody
    
    for row in Active_body: # gather data from each page
        Active_df_table['listing_id'].append(row.find_all('td', class_='d26m10')[0].get_text())
        Active_df_table['city'].append(row.find_all('td', class_='d26m12')[1].get_text())
        Active_df_table['address'].append(row.find_all('td', class_='d26m14')[0].get_text())
        Active_df_table['property_type'].append(row.find_all('td', class_='d26m12')[0].get_text())
        Active_df_table['price'].append(row.find_all('td', class_='d26m15')[0].get_text())
        Active_df_table['bed'].append(row.find_all('td', class_='d26m16')[0].get_text())
        Active_df_table['bath'].append(row.find_all('td', class_='d26m16')[1].get_text())
        Active_df_table['list_date'].append(row.find_all('td', class_='d26m5')[0].get_text())
        
        if row.td.span.a != 0 and page==0:
            Active_df_table['total_square_ft'].append(row.find_all('td', class_='d26m7')[2].get_text())
            Active_df_table['above_square_ft'].append(row.find_all('td', class_='d26m7')[1].get_text())
            Active_df_table['hoa'].append(row.find_all('td', class_='d26m7')[4].get_text())
            
        elif row.td.span.a != 0 and page>=0:
            Active_df_table['total_square_ft'].append(row.find_all('td', class_='d26m17')[1].get_text())
            Active_df_table['above_square_ft'].append(row.find_all('td', class_='d26m17')[0].get_text())
            Active_df_table['hoa'].append(row.find_all('td', class_='d26m17')[3].get_text())
            
    if (page+1)%7 == 1 and pages < (page+7):
        page_count = page_count/2
        
    elif (page+1) == 8:
        page_count += 2
      
    browser.find_by_xpath(f'//*[@id="m_upPaging"]/span/a[{page_count}]').click()
    time.sleep(2)

browser.quit()       

In [7]:
Active_df = pd.DataFrame(Active_df_table)
# Sometimes the data is returned uneven, this is to test and correct for it
# print(len(Active_df_table['listing_id'])) # Active_df_table['listing_id'].pop(0)
Active_df.head()

,listing_id,city,address,property_type,price,total_square_ft,above_square_ft,bed,bath,list_date,hoa
0,5536658,Denver,8359 Cherokee Street,Residential,"$479,700","2,030","1,015",5,2,05/29/2022,No
1,4142574,Colorado Springs,15025 Hilton Head Court,Residential,"$645,000","3,494","2,118",5,4,05/29/2022,Yes
2,7855800,Estes Park,540 W Elkhorn Avenue Unit# B2,Residential,"$349,000",588,588,1,1,05/29/2022,Yes
3,6793775,Salida,118 Mesa Drive,Residential,"$889,000","2,455","1,708",3,4,05/29/2022,No
4,7046988,Lyons,Cedar,Land,"$97,000",,,,,05/29/2022,Yes


### Google API, cross reference 

In [8]:
lat=[]
lng=[]

for house in Active_df.index:
    address = Active_df['address'][house].replace(' ','+')
    city = '+' + Active_df['city'][house].replace(' ','+')
    print(address+city)
    
    try:
        response = requests.get(
            f'https://maps.googleapis.com/maps/api/geocode/json?address={address},{city},+CO&key={api_key}')
        resp_json_payload = response.json()
        lat.append(resp_json_payload['results'][0]['geometry']['location']['lat'])
        lng.append(resp_json_payload['results'][0]['geometry']['location']['lng'])
        
    except:
        lat.append(np.nan)
        lng.append(np.nan)        
        print('address unavaible')

8359+Cherokee+Street+Denver
15025+Hilton+Head+Court+Colorado+Springs
540+W+Elkhorn+Avenue+Unit#+B2+Estes+Park
address unavaible
118+Mesa+Drive+Salida
+Cedar+Lyons
239+Pole+Creek+Drive+Tabernash
1011+Sycamore+Road+Crestone
1944+Alvarado+Road+Georgetown
422+S+Union+Boulevard+Lakewood
5852+Whale+Rock+Way+Morrison
4425+S+Price+Rd+Deer+Trail
796+Spruce+Drive+Twin+Lakes
222+Creekside+Drive+Unit#+316+Frisco
address unavaible
648+Poncho+Road+Larkspur
4554+Starboard+Drive+Boulder
2103+W+28th+Avenue+Denver
2868+S+Heather+Gardens+Way+Unit#+205+Aurora
address unavaible
2411+S+Xanadu+Way+Unit#+D+Aurora
address unavaible
1274+N+Tabor+Drive+Castle+Rock
Lot+9+Applewood+Drive+Colorado+City
29940+Fir+Drive+Evergreen
700+Sunset+Drive+Castle+Rock
12085+Sylvan+Meadows+Drive+Colorado+Springs
72+Corinthian+Circle+Unit#+104,+Bldg+D+Dillon
address unavaible
7675+Walker+Road+Colorado+Springs
20+Hunki+Dori+Court+Unit#+2220+Keystone
address unavaible
206+206+1/2+E+Park+Fowler
44100+E+County+Road+6+E+Bennett
4636+

30+Fairfax+Street+Denver
8700-8702+Richthofen+Place+Denver
4240+Falcon+Drive+Fort+Lupton
16682+Eolus+Way+Broomfield
12183+Arnett+Ranch+Road+Littleton
5793+Brennan+Avenue+Colorado+Springs
1900+S+Fillmore+Street+Denver
1650+Ingalls+Street+Lakewood
246+County+Road+640+Granby
1825+Kendall+Street+Unit#+105+Lakewood
address unavaible
8567+County+Road+3+Woodland+Park
1357+Steele+Street+Denver
828+Grand+Avenue+Unit#+201a+Grand+Lake
address unavaible
308+E+Magnolia+Street+Fort+Collins
900+Mountain+View+Avenue+Lot+103+Longmont
4501+Mirage+Drive+Castle+Rock
1748+Derbishire+Road+Jefferson
4601+County+Road+85+Tabernash
2026+Perry+Street+Denver
7823+County+Road+43+Bailey
3763+Bucknell+Drive+Highlands+Ranch
0953+Straight+Creek+Drive+Unit#+101,+Bldg+S+Dillon
address unavaible
10055+63rd+Avenue+Denver
6289+Taylor+Street+Frederick
8055+E+128th+Place+Thornton
16255+Thunder+Cat+Way+Monument
444+Ridge+Place+Woodland+Park
18701+E+Water+Drive+Unit#+E+Aurora
address unavaible
10075+63rd+Avenue+Denver
3075+S+S

57111+County+Road+Ll57+Villa+Grove
1877+S+Lincoln+Street+Denver
2200+Bellaire+Street+Denver
10795+County+Road+197a+Unit#+127+Nathrop
address unavaible
13945+Hudson+Street+Thornton
9330+Sun+Country+Drive+Elizabeth
883+Marfell+Street+Erie
10835+Beas+Drive+Conifer
264+Montezuma+Street+Brighton
5768+S+Malta+Street+Centennial
8142+S+Quatar+Circle+Aurora
2890+Monaco+Parkway+Denver
10604+Star+Thistle+Court+Highlands+Ranch
4923+Snowberry+Lane+Evergreen
2757+Grove+Street+Denver
1366+Garfield+Street+Unit#+105+Denver
address unavaible
78+Jackson+Street+Unit#+H+Denver
address unavaible
5248+Mount+Cutler+Court+Colorado+Springs
9835+W+Hinsdale+Place+Littleton
23454+E+Heritage+Parkway+Aurora
1988+S+Ouray+Street+Aurora
6072+S+Glencoe+Way+Centennial
10174+Fawnbrook+Lane+Highlands+Ranch
8240+Turnpike+Drive+Westminster
1026+Red+Oak+Drive+Longmont
2406+14th+Avenue+Greeley
4932+W+65th+Place+Arvada
2133+Primo+Road+Unit#+205+Highlands+Ranch
address unavaible
6205+Pulpit+Rock+Drive+Colorado+Springs
1293+S+Joh

1866+S+Corona+Street+Denver
19037+E+64th+Avenue+Denver
130+Dianna+Drive+Littleton
2251+N+Hooker+Street+Denver
4830+W+102nd+Avenue+Westminster
6139+Rocking+Chair+Lane+Unit#+L+Colorado+Springs
address unavaible
18262+E+Crestline+Circle+Centennial
7828+S+Hill+Circle+Littleton
10648+Ursula+Way+Commerce+City
155+Ranch+Lane+Buena+Vista
36+Isabelle+Court+Fairplay
11225+Colony+Row+Broomfield
6880+W+Mississippi+Avenue+Unit#+A+Lakewood
address unavaible
9400+E+Iliff+Avenue+Unit#+252+Denver
address unavaible
1500+Red+Sky+Court+Fort+Collins
11912+W+Quarles+Avenue+Littleton
5580+W+80th+Place+Unit#+32+Arvada
address unavaible
217+7th+Street+Hugo
635+Elati+Street+Denver
3891+Pecos+Trail+Castle+Rock
Lot+7+Akron+Street+Aurora
238+S+Downing+Street+Denver
395+S+Elm+Street+Keenesburg
1321+6th+Street+Fort+Lupton
1921+Basil+Street+Strasburg
3127+N+Elizabeth+Street+Denver
1162+Rockhurst+Drive+Unit#+307+Highlands+Ranch
address unavaible
6500+Roxborough+Park+Road+Littleton
13361+E+106th+Avenue+Commerce+City
16

5841+W+108th+Avenue+Westminster
1540+Tabor+Street+Lakewood
3535+Sedgewood+Way+Colorado+Springs
5553+Killarney+Court+Denver
2305+Talon+Parkway+Greeley
888+S+Reed+Court+Unit#+B+Lakewood
address unavaible
7653+Saulsbury+Street+Arvada
847+S+Logan+Street+Denver
2095+Monaco+Parkway+Denver
19167+E+54th+Place+Denver
155+Mount+Massive+Way+Longmont
+County+Road+U+Saguache
10627+Cherrybrook+Circle+Highlands+Ranch
4250+S+Olive+Street+Unit#+220+Denver
address unavaible
3304+W+127th+Avenue+Broomfield
3615+Kearney+Street+Denver
15903+E+Arkansas+Place+Aurora
8511+Essex+Drive+Denver
585+W+Jamison+Circle+Littleton
3435+Wyandot+Street+Denver
20156+E+Jefferson+Place+Aurora
240+S+Canosa+Court+Denver
789+Emporia+Street+Aurora
240+S+Canosa+Court+Denver
9735+Bucknell+Way+Highlands+Ranch
2910+Niagara+Street+Denver
436+S+3RD+Avenue+Deer+Trail
2825+Eliot+Street+Unit#+1+Denver
address unavaible
347+Silver+Cloud+Place+Castle+Rock
18705+W+84th+Place+Arvada
17470+E+Asbury+Circle+Aurora
7533+S+Sicily+Way+Aurora
745+S

133+Peppler+Drive+Longmont
1324+42nd+Avenue+Greeley
16096+Swan+Mountain+Drive+Broomfield
9126+Hunter+Street+Westminster
2746+Center+Park+Way+Berthoud
110+Noland+Court+Lyons
705+Collingswood+Drive+Fort+Collins
12674+Sunset+Drive+Firestone
219+5th+Avenue+Lyons
171+Penobscot+Court+Red+Feather+Lakes
983+Wedgwood+Road+Black+Hawk
543+Jicarilla+Trail+Red+Feather+Lakes
2919+Ptarmigan+Drive+Evans
1927+Reliance+Circle+Superior
7489+Mount+Meeker+Road+Longmont
413+Hickory+Lane+Johnstown
6881+S+Prince+Circle+Littleton
4452+County+Road+46+Johnstown
4420+Ann+Street+Fort+Collins
102+Birdie+Drive+Milliken
1425+Upland+Avenue+Boulder
1998+Foxtail+Lane+Unit#+B+Lafayette
address unavaible
27+Morton+Road+Red+Feather+Lakes
409+Terry+Street+Unit#+D+Longmont
address unavaible
817+Hemlock+Drive+Lyons
4606+Ipswich+Street+Boulder
121+E+Swallow+Road+Unit#+203+Fort+Collins
address unavaible
1109+Sunrise+Drive+Lochbuie
2862+Cascade+Creek+Drive+Lafayette
10031+Melody+Drive+Northglenn
11905+Blackfoot+Road+Conifer
804+

In [9]:
Active_df.insert(1, "lng", lng, True)
Active_df.insert(1, "lat", lat, True)
Active_df.to_csv('Resources/active_houses.csv', index_label="House number")

Active_df.head()

,listing_id,lat,lng,city,address,property_type,price,total_square_ft,above_square_ft,bed,bath,list_date,hoa
0,5536658,39.848180,-104.990981,Denver,8359 Cherokee Street,Residential,"$479,700","2,030","1,015",5,2,05/29/2022,No
1,4142574,39.049850,-104.820724,Colorado Springs,15025 Hilton Head Court,Residential,"$645,000","3,494","2,118",5,4,05/29/2022,Yes
2,7855800,NaN,NaN,Estes Park,540 W Elkhorn Avenue Unit# B2,Residential,"$349,000",588,588,1,1,05/29/2022,Yes
3,6793775,38.535829,-106.011878,Salida,118 Mesa Drive,Residential,"$889,000","2,455","1,708",3,4,05/29/2022,No
4,7046988,40.268444,-105.426859,Lyons,Cedar,Land,"$97,000",,,,,05/29/2022,Yes


### Map Vizualization

In [10]:
gmaps.configure(api_key=api_key)

In [11]:
Active_df = Active_df.dropna()

info_box_template = """
<dl>
<dt>Listing ID</dt><dd>{listing_id}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Price</dt><dd>{price}</dd>
<dt>Total Square Feet</dt><dd>{total_square_ft}</dd>
<dt>Bed & Bath</dt><dd>{bed}, {bath}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
home_info = [info_box_template.format(**row) for index, row in Active_df.iterrows()]
locations = Active_df[["lat", "lng"]]

In [12]:
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=home_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))